# Working with groups of rows of a data frame

In [ ]:
using DataFrames
using CSV
using Arrow
using Statistics
using FreqTables
using Pipe

using Test

# Adjust table display
ENV["LINES"]   = 15
ENV["COLUMNS"] = 200

# Make sure Chapter 3 is run
@test isfile("auto2.csv")

df = CSV.File("auto2.csv") |> DataFrame

In [ ]:
@test isfile("auto2.arrow")
df2 = Arrow.Table("auto2.arrow") |> DataFrame

In [ ]:
@test df == df2 # no missing in both tables

In general `Arrow.jl` is the preferred way to store data frames, but note that it uses its own `AbstractVector` type:

In [ ]:
typeof(df2.mpg)